In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm
import itertools
from collections import namedtuple, defaultdict, deque


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
input_text = au.read_txt_file_lines('input2.txt')
n_rows = len(input_text)
n_cols = len(input_text[0])
# for ii in range(1, n_rows):
#     assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 3 rows and 48 cols


In [54]:
data = []
machine = namedtuple('machine', ['con', 'but', 'act', 'mat', 'jol', 'arr'])

for l in input_text:
    con, tmp = l.split(']')
    buttons, jolts = tmp.split('{')

    con = np.array([1 if x == '#' else 0 for x in con.lstrip('[')])
    buttons = [tuple([int(k) for k in x.lstrip('(').rstrip(')').split(',')]) for x in buttons.split()]
    actions = [np.zeros(len(con), dtype=int) for _ in buttons]
    for i_b, b in enumerate(buttons):
        for i_c, c in enumerate(b):
            actions[i_b][c] = 1
    jolts = tuple([int(x) for x in jolts.rstrip('}').split(',')])
    mat = np.array(actions)
    arr = np.array(jolts)
    data.append(machine(con=con, but=buttons, act=actions, mat=mat, jol=jolts, arr=arr))

Sounds like a BFS search over graph .. ? Or Dijkstras [but with equal path lenght .?]?

we have the start and end node. 

In [61]:
# n_buttons = [len(m.but) for m in data]
# sorted(n_buttons), max(n_buttons)

In [62]:
def find_shortest_path(start, end, actions, mode='part1'):
    dict_visited = {tuple(start): 0}
    queue = deque([start])

    while len(queue) > 0:
        node = queue.popleft() 
        n_steps = dict_visited[tuple(node)]
        for a in actions:
            if mode == 'part1':
                new_node = np.abs(node - a)
            elif mode == 'part2':
                new_node = node + a

            if tuple(new_node) in dict_visited:
                continue 
            elif np.all(new_node == end):
                return n_steps + 1
            else:
                dict_visited[tuple(new_node)] = n_steps + 1
                queue.append(new_node)


total = 0
for m in tqdm(data):
    total +=find_shortest_path(np.zeros_like(m.con), m.con, m.act, mode='part1')
total
    




100%|██████████| 3/3 [00:00<00:00, 4419.71it/s]


7

# Part 2

In [63]:
# Naive BFS, takes too long!
total = 0
for m in tqdm(data):
    total += find_shortest_path(np.zeros_like(m.con), m.jol, m.act, mode='part2')
    print(total)
total

100%|██████████| 3/3 [00:00<00:00, 108.18it/s]

10
22
33


33

What's a smarter way .. 

the solution will be `jol = c1 * button_1 + c2 * button_2 .. `
(we can easily the find the max for each `c`, but still combinatiorially huge probably)
Let's break this down:

`jol_1 = c1 * b_1_1 + c2 * b_2_1 + ..`
`jol_2 = c1 * b_1_2 + c2 * b_2_2 + ..`

so we will have `len(buttons)` variables with `len(jolt)` equations. But there are many zeroes so probably it's solvable .. 

In [33]:
def lin_system_solver(mat, arr, verbose=0):
    assert type(mat) == np.ndarray, type(mat)
    assert type(arr) == np.ndarray 
    assert mat.ndim == 2 and arr.ndim == 1

    assert mat.shape[1] == len(arr)
    n_coef, n_arr = mat.shape 
    if verbose:
        print(f'There are {n_coef} coefs/buttons and {n_arr} jolts to match.')
        ## so coef[:] * mat[:, k] = arr[k]

        if n_arr == n_coef:
            print('solve with inv?')
        elif n_arr < n_coef:
            print('might be inf solutions')
        else:
            print('should be constrained')

    coefs = np.zeros(n_coef, dtype=int) - 1  ## should be >= 0 in the end
    # coefs = np.round(lstsq(np.array(m.act).T, np.array(m.jol))[0]).astype(int)

    ## For each coef, get list of relevant buttons. 
    ##      If 1 button; coef solved.
    ## For coef 1, use first button. Thereafter, use first available button. 
    ## subtract the coef from all other coefs -> effectively changes list of buttons per coef. 

   
    coefs_solved = {}
    for k in range(n_arr):
        if np.sum(mat[:, k]) == 1:
            r = np.where(mat[:, k])[0]
            coefs_solved[int(r)] = int(arr[k])

    print(coefs_solved)
   

    change = True 
    while len(coefs_solved) < n_coef:
        if change == False:
            print('Failed')
            break
        change = False
        for k, j in enumerate(arr):
            tmp_coefs = np.where(mat[:, k])[0]
            tmp_coefs_sel = [x for x in tmp_coefs if x not in coefs_solved]
            if len(tmp_coefs_sel) == 1:
                new_c = int(tmp_coefs_sel[0])
                coefs_solved[new_c] = int(j - sum([coefs_solved[r] for r in tmp_coefs if r != new_c]))
                change = True 
        
   
    # inds_solved = set()
    # dict_solves = {}
    # for k, j in enumerate(arr):
    #     inds_nonzero_c = [int(x) for x in np.where(mat[:, k])[0]]
    #     print(inds_nonzero_c)

    #     ind_solve = None
    #     while len(inds_nonzero_c) > 0:
    #         i_c = inds_nonzero_c.pop()
    #         if i_c not in inds_solved:
    #             ind_solve = i_c 
    #             inds_solved.add(ind_solve)
    #             print(inds_solved)
    #             print(f'Solving coef {ind_solve} for arr {k} with value {j}.')
    #             break 
        
    #     dict_solves[ind_solve] = (int(j), k, inds_nonzero_c)
    
    # coefs_solved = {}

    # for c, v in dict_solves.items():
    #     assert c not in coefs_solved, c
    #     j, remainders = v 
    #     if len(remainders) == 0:
    #         coefs_solved[c] = int(j)
    #         continue 

    #     solvable = True 
    #     for r in remainders:
    #         if r not in coefs_solved:
    #             solvable = False 
        # if solvable:
        #     coefs_solved = j -  

    return coefs_solved








    for k, j in enumerate(arr):
        # assert j - coefs[mat[:, k].astype('bool')].sum() == 0, (k, j, coefs)
        if j - coefs[mat[:, k].astype('bool')].sum() != 0 or np.min(coefs) < 0:
            return None
    return coefs

# m = data[0]
total = 0
for m in data:
    coefs = lin_system_solver(m.mat, m.arr)
    total += sum(coefs.values())
print(total)
# total = 0
# m_failed = []
# for m in data:
#     coefs = lin_system_solver(m.mat, m.arr)
#     if coefs is None:
#         m_failed.append(m)
#     else:
#         total += np.sum(coefs)

# print(f'Total: {total}. Failed: {len(m_failed)}/{len(data)}')

{}
Failed
{}
Failed
{2: 5}
Failed
5


/var/folders/kh/y9z_055x6gq8gc74b6y16r1c0000gn/T/ipykernel_52850/391327184.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  coefs_solved[int(r)] = int(arr[k])


In [65]:
from time import sleep

def fun(x):
    sleep(0.1)
    return x

%timeit fun(3)

104 ms ± 575 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
m.mat, m.arr

(array([[1, 1, 1, 1, 1, 0],
        [1, 0, 0, 1, 1, 0],
        [1, 1, 1, 0, 1, 1],
        [0, 1, 1, 0, 0, 0]]),
 array([10, 11, 11,  5, 10,  5]))

In [19]:
m.mat[:, 0], m.arr

(array([0, 1, 0, 0]), array([ 5, 38, 18, 20,  5, 14]))

In [ ]:
# Naive BFS, takes too long!
# total = 0
for m in tqdm(m_failed):
    total += find_shortest_path(np.zeros_like(m.con), m.jol, m.act, mode='part2')
    # print(total)
total

In [42]:
def lin_system_solver2(mat, arr, verbose=0):
    assert type(mat) == np.ndarray, type(mat)
    assert type(arr) == np.ndarray 
    assert mat.ndim == 2 and arr.ndim == 1
    arr = copy.deepcopy(arr)
    assert mat.shape[1] == len(arr)
    n_coef, n_arr = mat.shape 
    if verbose:
        print(f'There are {n_coef} coefs/buttons and {n_arr} jolts to match.')
        ## so coef[:] * mat[:, k] = arr[k]

        if n_arr == n_coef:
            print('solve with inv?')
        elif n_arr < n_coef:
            print('might be inf solutions')
        else:
            print('should be constrained')



    ## Greedy approach: start with the coef that affects the most arrs.

    # coefs_solved = {}
    # for k in range(n_arr):
    #     if np.sum(mat[:, k]) == 1:
    #         r = np.where(mat[:, k])[0]
    #         coefs_solved[int(r)] = int(arr[k])
    # coefs_sorted = sorted([(int(mat[c, :].sum()), c) for c in range(n_coef)])[::-1]
    # for sum_ones, c in coefs_sorted:
    #     min_val = max(int(arr[mat[c, :].astype('bool')].min()), 0)
    #     # print(c, coefs_solved, arr, min_val)
    #     if int(c) in coefs_solved:
    #         # assert coefs_solved[int(c)] == min_val, coefs_solved
    #         coefs_solved[int(c)] = min(min_val, coefs_solved[int(c)])
    #     coefs_solved[int(c)] = min_val
    #     arr[mat[c, :].astype('bool')] -= min_val 
    #     assert np.all(arr[mat[c, :]] >= 0), arr

    # sum_coefs = sum(coefs_solved.values())
    # return coefs_solved, sum_coefs + np.sum(arr)

    ## Full sweep:

    # coefs_sorted = sorted([(int(mat[c, :].sum()), c) for c in range(n_coef)])[::-1]
    # for sum_ones, c in coefs_sorted:
    best_est = np.inf
    for coefs_sorted in itertools.permutations(range(n_coef), n_coef):
        tmp_arr = copy.deepcopy(arr)
        tmp_coefs = np.zeros(n_coef)
        for c in coefs_sorted:
            c = int(c)
            min_val = max(int(arr[mat[c, :].astype('bool')].min()), 0)
            tmp_coefs[c] = min_val
            tmp_arr[mat[c, :].astype('bool')] -= min_val
        print(tmp_coefs, coefs_sorted)
        if np.all(tmp_coefs @ mat - arr == 0):
            return np.sum(tmp_coefs)

    return False

    # coefs_arr = np.zeros(n_coef)
    # for i_c, c in coefs_solved.items():
    #     coefs_arr[i_c] = c 
    # shortage = 0 

    # print(coefs)
    # print(mat[:, k].astype('bool'))

    # for k, j in enumerate(arr):
    #     diff = j - coefs_arr[mat[:, k].astype('bool')].sum()
    #     if diff != 0: 
    #         shortage += diff 
    #         print(k, j, diff)
    # print(np.sum(arr))

    

# total = 0
# for m in data:
#     sum_coefs = lin_system_solver2(m.mat, m.arr)
#     print(sum_coefs, m.mat, m.arr)
#     total += sum_coefs
# print(total)


In [43]:
m = data[0]
lin_system_solver2(m.mat, m.arr)

[18.  5. 20. 14.] (0, 1, 2, 3)
[18.  5. 20. 14.] (0, 1, 3, 2)
[18.  5. 20. 14.] (0, 2, 1, 3)
[18.  5. 20. 14.] (0, 2, 3, 1)
[18.  5. 20. 14.] (0, 3, 1, 2)
[18.  5. 20. 14.] (0, 3, 2, 1)
[18.  5. 20. 14.] (1, 0, 2, 3)
[18.  5. 20. 14.] (1, 0, 3, 2)
[18.  5. 20. 14.] (1, 2, 0, 3)
[18.  5. 20. 14.] (1, 2, 3, 0)
[18.  5. 20. 14.] (1, 3, 0, 2)
[18.  5. 20. 14.] (1, 3, 2, 0)
[18.  5. 20. 14.] (2, 0, 1, 3)
[18.  5. 20. 14.] (2, 0, 3, 1)
[18.  5. 20. 14.] (2, 1, 0, 3)
[18.  5. 20. 14.] (2, 1, 3, 0)
[18.  5. 20. 14.] (2, 3, 0, 1)
[18.  5. 20. 14.] (2, 3, 1, 0)
[18.  5. 20. 14.] (3, 0, 1, 2)
[18.  5. 20. 14.] (3, 0, 2, 1)
[18.  5. 20. 14.] (3, 1, 0, 2)
[18.  5. 20. 14.] (3, 1, 2, 0)
[18.  5. 20. 14.] (3, 2, 0, 1)
[18.  5. 20. 14.] (3, 2, 1, 0)


False

In [50]:
np.array([18, 5, 20, 14]) @ m.mat, m.arr

(array([ 5, 52, 18, 34,  5, 14]), array([ 5, 38, 18, 20,  5, 14]))

In [48]:
m.mat * np.array([18, 5, 20, 14])[:, None]

array([[ 0, 18, 18,  0,  0,  0],
       [ 5,  0,  0,  0,  5,  0],
       [ 0, 20,  0, 20,  0,  0],
       [ 0, 14,  0, 14,  0, 14]])

In [49]:
np.sum(m.mat * np.array([18, 5, 20, 14])[:, None], 0)

array([ 5, 52, 18, 34,  5, 14])

In [ ]:
def lin_system_solver3(mat, arr, verbose=0):
    assert type(mat) == np.ndarray, type(mat)
    assert type(arr) == np.ndarray 
    assert mat.ndim == 2 and arr.ndim == 1
    arr = copy.deepcopy(arr)
    assert mat.shape[1] == len(arr)
    n_coef, n_arr = mat.shape 
    if verbose:
        print(f'There are {n_coef} coefs/buttons and {n_arr} jolts to match.')
        ## so coef[:] * mat[:, k] = arr[k]

        if n_arr == n_coef:
            print('solve with inv?')
        elif n_arr < n_coef:
            print('might be inf solutions')
        else:
            print('should be constrained')

    best_est = np.inf
    for coefs_sorted in itertools.permutations(range(n_coef), n_coef):
        tmp_arr = copy.deepcopy(arr)
        tmp_coefs = np.zeros(n_coef)
        for c in coefs_sorted:
            c = int(c)
            min_val = max(int(arr[mat[c, :].astype('bool')].min()), 0)
            tmp_coefs[c] = min_val
            tmp_arr[mat[c, :].astype('bool')] -= min_val
        print(tmp_coefs, coefs_sorted)
        if np.all(tmp_coefs @ mat - arr == 0):
            return np.sum(tmp_coefs)

    return False



In [59]:
m = data[0]
m.mat, m.arr

(array([[0, 0, 0, 1],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [1, 0, 1, 0],
        [1, 1, 0, 0]]),
 array([3, 5, 4, 7]))

In [56]:
data

[machine(con=array([0, 1, 1, 0]), but=[(3,), (1, 3), (2,), (2, 3), (0, 2), (0, 1)], act=[array([0, 0, 0, 1]), array([0, 1, 0, 1]), array([0, 0, 1, 0]), array([0, 0, 1, 1]), array([1, 0, 1, 0]), array([1, 1, 0, 0])], mat=array([[0, 0, 0, 1],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [1, 0, 1, 0],
        [1, 1, 0, 0]]), jol=(3, 5, 4, 7), arr=array([3, 5, 4, 7])),
 machine(con=array([0, 0, 0, 1, 0]), but=[(0, 2, 3, 4), (2, 3), (0, 4), (0, 1, 2), (1, 2, 3, 4)], act=[array([1, 0, 1, 1, 1]), array([0, 0, 1, 1, 0]), array([1, 0, 0, 0, 1]), array([1, 1, 1, 0, 0]), array([0, 1, 1, 1, 1])], mat=array([[1, 0, 1, 1, 1],
        [0, 0, 1, 1, 0],
        [1, 0, 0, 0, 1],
        [1, 1, 1, 0, 0],
        [0, 1, 1, 1, 1]]), jol=(7, 5, 12, 7, 2), arr=array([ 7,  5, 12,  7,  2])),
 machine(con=array([0, 1, 1, 1, 0, 1]), but=[(0, 1, 2, 3, 4), (0, 3, 4), (0, 1, 2, 4, 5), (1, 2)], act=[array([1, 1, 1, 1, 1, 0]), array([1, 0, 0, 1, 1, 0]), array([1, 1, 1, 0, 1, 1]), array([0

In [ ]:
def lin_system_solver4(mat, arr, verbose=0):
    assert type(mat) == np.ndarray, type(mat)
    assert type(arr) == np.ndarray 
    assert mat.ndim == 2 and arr.ndim == 1
    arr = copy.deepcopy(arr)
    assert mat.shape[1] == len(arr)
    n_coef, n_arr = mat.shape 
    if verbose:
        print(f'There are {n_coef} coefs/buttons and {n_arr} jolts to match.')
        ## so coef[:] * mat[:, k] = arr[k]

        if n_arr == n_coef:
            print('solve with inv?')
        elif n_arr < n_coef:
            print('might be inf solutions')
        else:
            print('should be constrained')

    
    CoefEq = namedtuple('CoefEq', ['vars', 'inds_nz', 'val'])
    for i_c in range(n_arr):
        vars = mat[i_c, :]
        inds_nz = np.where(vars == 1)[0]
        val = arr[i_c]



    return False



In [64]:
m = data[0] 

m.mat, m.arr

(array([[0, 0, 0, 1],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [1, 0, 1, 0],
        [1, 1, 0, 0]]),
 array([3, 5, 4, 7]))